# Hurricane Risk Visualization

This visualization uses a gpu to plot households and tanks on a dashboard, with range slider features to show households that have a certain risk index range. Also we have included the elderly multiselect, and household/tank distance slider.

### Import Statements

In [1]:
import geopandas as gpd

import cuxfilter
from cuxfilter.layouts import feature_and_five_edge, double_feature
import cudf
import numpy as np

import holoviews as hv
import pandas as pd

from pyproj import Proj, Transformer



### Importing Hurricane Risk and Distance Data

This is a preprocessed parquet file that includes the latitude and longitude of households and tanks that are already transformed into the 3857 projections. This file also contains the ```is_elderly``` variable that indicates if the point at a particular lat/lon is a household with elderly, a household without elderly, or a tank. ```distance_mi``` is the distance in miles between tanks and houses. 

In [13]:
df = pd.read_parquet('/hpc/group/codeplus22-vis/infousa_copy/hurricane_risks_final.parquet')
df

is_elderly  distance_mi  hurricane_risk      lat_3857      lon_3857
0                  2    33.459368        5.380037 -8.509454e+06  5.101307e+06
1                  2    28.501947        4.895073 -8.499018e+06  5.096218e+06
2                  2    28.592896        4.895073 -8.496356e+06  5.099448e+06
3                  2    33.876175        5.380037 -8.509963e+06  5.103102e+06
4                  2    33.117674        5.380037 -8.508370e+06  5.099066e+06
...              ...          ...             ...           ...           ...
53165525           0   215.000000       -1.000000 -1.167972e+07  4.833652e+06
53165526           0   215.000000       -1.000000 -1.167971e+07  4.833633e+06
53165527           0   215.000000       -1.000000 -1.167971e+07  4.833691e+06
53165528           0   215.000000       -1.000000 -1.167971e+07  4.833535e+06
53165529           0   215.000000       -1.000000 -1.167972e+07  4.833535e+06

[53165530 rows x 5 columns]

### Transforming the dataframe type

Here, we are transforming the pandas dataframe into a cudf dataframe and then into a cuxfilter dataframe so that we can plot the points using the cuxfilter library.

In [14]:
cdf = cudf.DataFrame.from_pandas(df) 

In [15]:
cux_df = cuxfilter.DataFrame.from_dataframe(cdf) 

### Creating multiselect label map and defining map color palette

In the label map for the ```is_elderly``` multiselect, we are defining the labels for the values that equal 0, 1, and 2 respectively.

In [16]:
label_elderly = {0: 'Tank', 1: 'Elderly', 
             2: 'No Elderly'}

from bokeh.palettes import Spectral11
colors = list(Spectral11)
colors.reverse()


### Defining charts

Below, we are defining the four charts that will be plotted on our dashboard. In each chart, you can define the type of cuxfilter chart (```scatter```, ```multiselect```, ```range_slider```, ```drop_down```, etc.) and also specify the column that the chart is made on. The label map parameter specifies which label map the feature will pull from. 

In [20]:
household = cuxfilter.charts.scatter(x='lat_3857', y='lon_3857', pixel_shade_type='linear', color_palette = colors, aggregate_fn = 'max', aggregate_col = 'distance_mi', tile_provider="CartoDark", title = 'Households and distances to nearest tank, with hurricane risk slider', legend = True)

elderly = cuxfilter.charts.multi_select('is_elderly', label_map = label_elderly)

hurricane_risk= cuxfilter.charts.range_slider('hurricane_risk')

dist = cuxfilter.charts.range_slider('distance_mi')


### Plotting the charts on the dashboard

The layout and sidebar parameter determines how the charts are oganized on the dashboard. Here we have put three charts as the sidebar, however, you could also place those three charts underneath the main chart by specifying feature and base under the layout feature.

In [21]:
d = cux_df.dashboard([household], sidebar = [elderly, hurricane_risk, dist], layout = cuxfilter.layouts.feature_and_base, theme = cuxfilter.themes.rapids) 


### Display the visualization

In [19]:
d.show()
d.app(sidebar_width = 250)

Dashboard running at port 41895


Row(sizing_mode='stretch_both')
    [0] Column(sizing_mode='stretch_height', width=250)
        [0] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='Datapoints Selected')
            [0] Column(sizing_mode='stretch_both')
                [0] Number(css_classes=['indicator'], font_size='18pt', format='{value:,}', sizing_mode='stretch_width', value=53165530)
                [1] Progress(sizing_mode='stretch_width', value=100)
        [1] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='is_elderly_multi_select', width=400)
            [0] MultiSelect(height=200, options={'Tank': 0, 'Elderly': 1, ...}, sizing_mode='stretch_both', value=[''], width=400)
        [2] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='hurricane_risk_range_slid..., width=400)
            [0] RangeSlider(end=100.0, sizing_mode='stretch_both', start=-1.0, value=(-1.0, 100.0), value_end=100.0, value_start=-1.0)
        [3] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='distance_mi_range_slider'..., width=400)
            [0] RangeSlider(end=215.0, sizing_mode='stretch_both', start=0.000592927239086057, value=(0.000592927239086057, ..., value_end=215.0, value_start=0.000592927239086057)
    [1] GridStack(allow_drag=False, allow_resize=False, ncols=11, nrows=4, sizing_mode='stretch_both')
        [0] Card(sizing_mode='stretch_both', title='Households and distances ..., width=800)
            [0] HoloViews(DynamicMap, height=400, sizing_mode='stretch_both')